# Piaui Herald - Exploratory Data Analysis
Finding interesting cases for Rosie's column

In [1]:
import numpy as np
import pandas as pd
from re import compile, search
from datetime import timedelta

In [2]:
dataset = pd.read_csv('../../../serenata-data/2017-03-15-reimbursements.xz',
                      dtype={'applicant_id': np.str,
                             'cnpj_cpf': np.str,
                             'congressperson_id': np.str,
                             'subquota_number': np.str,
                             'document_id': np.int},
                      low_memory=False)

## Luxury Hotel

We are aiming to find suspicious expenses in hotels, maybe someone spent the holidays in some luxury hotel and asked for reimbursement

In [3]:
lodging = dataset[dataset['subquota_description'] == 'Lodging, except for congressperson from Distrito Federal']
keys = ['applicant_id','congressperson_id','cnpj_cpf', 'supplier']
grouped = lodging.groupby(keys)

Number of Lodging expenses

In [4]:
len(grouped)

21064

In [5]:
subquota_numbers = grouped['subquota_number'].agg(lambda x: ','.join(x)).reset_index()
subquota_numbers.head()

,applicant_id,congressperson_id,cnpj_cpf,supplier,subquota_number
0,1001,73768,01044236000192,ORION HOTEL LTDA,14
1,1001,73768,01233159000119,GUAMIRANGA HOTEL,14
2,1001,73768,01233159000119,GUAMIRANGA HOTEL LTDA,"14,14"
3,1001,73768,01680970000148,Hotel e churrascaria do Gaúcho,14
4,1001,73768,01687173000192,BOA VISTA HOTEL LTDA.,14


In [6]:
document_ids = grouped['document_id'].agg(lambda x: tuple(x)).reset_index()
document_ids.head()

,applicant_id,congressperson_id,cnpj_cpf,supplier,document_id
0,1001,73768,01044236000192,ORION HOTEL LTDA,"(2277751,)"
1,1001,73768,01233159000119,GUAMIRANGA HOTEL,"(1901299,)"
2,1001,73768,01233159000119,GUAMIRANGA HOTEL LTDA,"(1901305, 1901309)"
3,1001,73768,01680970000148,Hotel e churrascaria do Gaúcho,"(5324333,)"
4,1001,73768,01687173000192,BOA VISTA HOTEL LTDA.,"(5100425,)"


In [7]:
net_values_sum = grouped['total_net_value'].agg({'sum': np.sum}).reset_index()
net_values_sum.head()

,applicant_id,congressperson_id,cnpj_cpf,supplier,sum
0,1001,73768,01044236000192,ORION HOTEL LTDA,99.3
1,1001,73768,01233159000119,GUAMIRANGA HOTEL,40.0
2,1001,73768,01233159000119,GUAMIRANGA HOTEL LTDA,99.0
3,1001,73768,01680970000148,Hotel e churrascaria do Gaúcho,90.0
4,1001,73768,01687173000192,BOA VISTA HOTEL LTDA.,129.0


In [8]:
aggregation = pd.merge(pd.merge(subquota_numbers, document_ids, on=keys),
                       net_values_sum, on=keys)
aggregation.head()

,applicant_id,congressperson_id,cnpj_cpf,supplier,subquota_number,document_id,sum
0,1001,73768,01044236000192,ORION HOTEL LTDA,14,"(2277751,)",99.3
1,1001,73768,01233159000119,GUAMIRANGA HOTEL,14,"(1901299,)",40.0
2,1001,73768,01233159000119,GUAMIRANGA HOTEL LTDA,"14,14","(1901305, 1901309)",99.0
3,1001,73768,01680970000148,Hotel e churrascaria do Gaúcho,14,"(5324333,)",90.0
4,1001,73768,01687173000192,BOA VISTA HOTEL LTDA.,14,"(5100425,)",129.0


Get net value by row

In [9]:
def get_top_net_value(row):
    l = list(row['document_id'])
    values = []
    for reimbursement_id in l:
        values.append(float(dataset[dataset['document_id'] == reimbursement_id]['total_net_value']))
    return {'top_net_value':max(values), 'top_document':l[values.index(max(values))]}

top_things = aggregation.apply(func=get_top_net_value, axis='columns')
aggregation['top_net_value'], aggregation['top_document'] = "",""

for _ in range(len(top_things)):
    # paliative since DataFrame.replace() did not work ¯\_(ツ)_/¯
    aggregation.loc[_, 'top_net_value'] = top_things[_]['top_net_value']
    aggregation.loc[_, 'top_document'] = top_things[_]['top_document']

aggregation = aggregation.sort_values(by='top_net_value', ascending=False)
aggregation.head(10)

,applicant_id,congressperson_id,cnpj_cpf,supplier,subquota_number,document_id,sum,top_net_value,top_document
15494,2389,160633,07959819000120,KAPITAL VIAG. E TUR. LTDA,"14,14,14,14,14,14","(5877120, 5952491, 5952505, 5988139, 6025233, ...",24842.08,11090.1,5877120
20048,80,74319,04443021000213,EUROPA INVESTIMENTOS LTDA,"14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,1...","(5037717, 5054130, 5054141, 5054148, 5072351, ...",139915.00,10665,5322461
14774,2362,160606,05668014000100,D.A.N HOTEL SP LTDA,"14,14,14,14,14,14,14,14,14,14,14,14,14,14,14","(2301718, 2385930, 2414282, 2443857, 5058637, ...",53664.07,10170,5652142
15653,2392,160549,40848905000130,WM TURISMO E VIAGENS LTDA.,14,"(2349567,)",9316.20,9316.2,2349567
19983,775,73437,58133323000133,VASCO DA GAMA TURISMO LTDA,"14,14","(5168791, 5487203)",16000.00,9000,5487203
18078,2976,178981,61223608000170,MEGTUR AGENCIA DE VIAGENS E TURISMO LTDA,14,"(5794903,)",8510.00,8510,5794903
20051,80,74319,04863492000108,RIO POTY HOTEL,"14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,1...","(1643367, 1643368, 1839328, 1839336, 1867172, ...",61400.00,8050,1955426
17868,2955,178847,16978175000108,Adria viagens e turismo LTDA ME,"14,14,14,14,14,14,14,14,14,14,14,14","(5677346, 5677362, 5781905, 5781908, 5803661, ...",34425.02,7904.84,5884760
19616,611,74207,04433548000186,EMILIANO EMPREENDIMENTOS E PARTICIPAÇÕES HOTEL...,"14,14,14,14,14,14","(5150333, 5237164, 5346066, 5443587, 5531077, ...",32982.39,7697.22,5443587
955,1119,74177,34776559000120,NOVO HOTEL,"14,14,14,14,14,14,14,14,14,14","(5207833, 5349869, 5366314, 5410698, 5467753, ...",32144.00,7600,5497082


## Top eaters

Who were the congresspeople that ate more in one day and when?

In [10]:
meals = dataset[dataset['subquota_description'] == 'Congressperson meal']
meals = meals.reset_index()

In [11]:
keys = ['applicant_id','congressperson_name', 'issue_date']
meals_aggregation = meals.groupby(keys)['total_net_value']. \
                        agg({'sum': np.sum, 'expenses': len, 'mean': np.mean})
meals_aggregation['expenses'] = meals_aggregation['expenses'].astype(np.int)

In [12]:
meals_agg_top20 = meals_aggregation.sort_values(['expenses', 'sum'], ascending=[False, False]).head(20)
meals_agg_top20 = pd.DataFrame(meals_agg_top20.reset_index())
meals_agg_top20.loc[:,'issue_date'] = pd.to_datetime(meals_agg_top20['issue_date'], errors='coerce')

Bellow you can see the top 20 congresspeople that requested more reimbursements in one day

In [13]:
meals_agg_top20.head(20)

,applicant_id,congressperson_name,issue_date,sum,expenses,mean
0,1798,CELSO MALDANER,2011-09-05,750.28,13,57.713846
1,1862,JOSÉ PAULO TÓFFANO,2010-04-27,558.07,13,42.928462
2,1689,SANDRA ROSADO,2012-01-12,333.40,12,27.783333
3,1689,SANDRA ROSADO,2012-01-17,287.43,12,23.952500
4,1689,SANDRA ROSADO,2012-01-06,281.75,12,23.479167
5,1872,LÉO VIVAS,2010-08-31,630.00,11,57.272727
6,1689,SANDRA ROSADO,2012-01-11,541.56,11,49.232727
7,2257,PAULO WAGNER,2011-07-21,537.66,11,48.878182
8,1689,SANDRA ROSADO,2015-01-07,396.60,11,36.054545
9,1689,SANDRA ROSADO,2012-01-15,295.58,11,26.870909


In [14]:
def create_jarbas_consult_congressperson_meal(row):
    d = timedelta(days=1)
    jarbas_api = "https://jarbas.serenatadeamor.org/#/applicantId"
    link = [jarbas_api, row['applicant_id'],
         'issueDateEnd', (row['issue_date'] + d).strftime('%Y-%m-%d'),
         'issueDateStart', row['issue_date'].strftime('%Y-%m-%d'),
         'subquotaId', '13']
    return '/'.join(link)

meals_agg_top20 = meals_agg_top20.assign(link_jarbas_consult=meals_agg_top20. \
                                         apply(func=create_jarbas_consult_congressperson_meal, axis=1))

print('Here are the links for those top 20')

for _ in range(20):
    print(meals_agg_top20.iloc[_]['link_jarbas_consult'])

Here are the links for those top 20
https://jarbas.serenatadeamor.org/#/applicantId/1798/issueDateEnd/2011-09-06/issueDateStart/2011-09-05/subquotaId/13
https://jarbas.serenatadeamor.org/#/applicantId/1862/issueDateEnd/2010-04-28/issueDateStart/2010-04-27/subquotaId/13
https://jarbas.serenatadeamor.org/#/applicantId/1689/issueDateEnd/2012-01-13/issueDateStart/2012-01-12/subquotaId/13
https://jarbas.serenatadeamor.org/#/applicantId/1689/issueDateEnd/2012-01-18/issueDateStart/2012-01-17/subquotaId/13
https://jarbas.serenatadeamor.org/#/applicantId/1689/issueDateEnd/2012-01-07/issueDateStart/2012-01-06/subquotaId/13
https://jarbas.serenatadeamor.org/#/applicantId/1872/issueDateEnd/2010-09-01/issueDateStart/2010-08-31/subquotaId/13
https://jarbas.serenatadeamor.org/#/applicantId/1689/issueDateEnd/2012-01-12/issueDateStart/2012-01-11/subquotaId/13
https://jarbas.serenatadeamor.org/#/applicantId/2257/issueDateEnd/2011-07-22/issueDateStart/2011-07-21/subquotaId/13
https://jarbas.serenatadeamo

### Priciest Meals
What was the highest meal reimbursement, where it was made and by who?

Here are the top 20 priciest meals ever reimbursed.

In [15]:
priciest_meals = meals[['document_id', 'issue_date',
                        'total_net_value', 'congressperson_name',
                        'supplier']]. \
                            sort_values('total_net_value', ascending=False).head(20)
priciest_meals.head(20)

,document_id,issue_date,total_net_value,congressperson_name,supplier
151919,5857053,2015-11-25 00:00:00.0,6205.00,LIDERANÇA DO PT,CENTRAL PARK RESTAURANTE E EVENTOS LTDA ME
148474,5588112,2015-01-18 00:00:00.0,5880.00,MARLLOS SAMPAIO,VR COMERCIO DE VEICULOS LTDA-ME
1628,1689682,2009-09-01 00:00:00.0,5852.00,MANOEL SALVIANO,HOTEL NACIONAL S/A
84828,2383218,2012-08-22 00:00:00.0,5696.00,LIDERANÇA DO PT,PARRILLA FIGUEIRA DA VILLA
182594,5954206,2016-03-30T00:00:00,5142.25,LIDERANÇA DO PT,CENTRAL PARK RESTAURANTE E EVENTOS LTDA ME
151914,5834088,2015-11-03 00:00:00.0,5054.99,LIDERANÇA DO PT,CENTRAL PARK RESTAURANTE E EVENTOS LTDA ME
132370,5573885,2014-12-10 00:00:00.0,5050.00,LIDERANÇA DO PT,CENTRAL PARK RESTAURANTE E EVENTOS LTDA ME
109870,5056362,2013-03-18 00:00:00.0,4990.00,LIDERANÇA DO PSDB,Emporio Albamonte Comercio e Serviços Alimentí...
1631,1717750,2009-11-03 00:00:00.0,4840.00,MANOEL SALVIANO,HOTEL NACIONAL S/A
36007,2195568,2011-11-02 00:00:00.0,4510.00,SALVADOR ZIMBALDI,HOTEL NACIONAL SA


In [16]:
def create_jarbas_consult_document_id(row):
    jarbas_api = "https://jarbas.serenatadeamor.org/#/documentId"
    link = [jarbas_api, str(row['document_id'])]
    return '/'.join(link)

In [17]:
priciest_meals = priciest_meals.assign(link_jarbas_consult=priciest_meals. \
                                         apply(func=create_jarbas_consult_document_id, axis=1))

print('Here are the links for those top 20')

for _ in range(20):
    print(priciest_meals.iloc[_]['link_jarbas_consult'])

Here are the links for those top 20
https://jarbas.serenatadeamor.org/#/documentId/5857053
https://jarbas.serenatadeamor.org/#/documentId/5588112
https://jarbas.serenatadeamor.org/#/documentId/1689682
https://jarbas.serenatadeamor.org/#/documentId/2383218
https://jarbas.serenatadeamor.org/#/documentId/5954206
https://jarbas.serenatadeamor.org/#/documentId/5834088
https://jarbas.serenatadeamor.org/#/documentId/5573885
https://jarbas.serenatadeamor.org/#/documentId/5056362
https://jarbas.serenatadeamor.org/#/documentId/1717750
https://jarbas.serenatadeamor.org/#/documentId/2195568
https://jarbas.serenatadeamor.org/#/documentId/5104194
https://jarbas.serenatadeamor.org/#/documentId/1751733
https://jarbas.serenatadeamor.org/#/documentId/5760917
https://jarbas.serenatadeamor.org/#/documentId/1689685
https://jarbas.serenatadeamor.org/#/documentId/1714197
https://jarbas.serenatadeamor.org/#/documentId/5736053
https://jarbas.serenatadeamor.org/#/documentId/2360622
https://jarbas.serenatadeamor

## Roasted goat
Some congress people seem to be very fond of roasted goat

In [18]:
len(meals)

197237

In [19]:
bode = compile('BODE')
bode_bool = []
for _ in range(len(meals)):
    if bode.search(meals.loc[_, 'supplier']):
        bode_bool.append(True)
    else:
        bode_bool.append(False)

In [20]:
bode_meals = meals[bode_bool]
keys = ['congressperson_id','cnpj_cpf', 'supplier', 'issue_date', 'document_id']
bode_meals_grouped = bode_meals.groupby(keys)

bode_meals_grouped = bode_meals_grouped['total_net_value'].agg({'expense': np.sum}).reset_index()
bode_meals_grouped = bode_meals_grouped.sort_values('expense', ascending=False)
bode_meals_top20 = bode_meals_grouped.head(20)
bode_meals_top20.head(20)

,congressperson_id,cnpj_cpf,supplier,issue_date,document_id,expense
250,74419,03487767000185,GERALDO BODE ASSADO,2011-02-13 00:00:00.0,2006450,1493.00
249,74419,03487767000185,GERALDO BODE ASSADO,2010-08-02 00:00:00.0,1905263,480.00
240,74419,03487767000185,GERALDO BODE ASSADO,2010-06-18 00:00:00.0,1882756,440.00
529,74419,16814703000193,BODEGA DA SERRA,2014-07-21 00:00:00.0,5476790,250.00
516,74419,03487767000185,GERALDO BODE ASSADO,2012-01-07 00:00:00.0,2249894,245.00
252,74419,03487767000185,GERALDO BODE ASSADO,2012-01-15 00:00:00.0,2249881,243.00
277,74419,03487767000185,GERALDO BODE ASSADO,2013-03-23 00:00:00.0,5061745,240.00
515,74419,03487767000185,GERALDO BODE ASSADO,2012-01-02 00:00:00.0,2249896,210.00
264,74419,03487767000185,GERALDO BODE ASSADO,2012-08-04 00:00:00.0,2385459,210.00
164,163322,03459794000144,ENTRE AMIGOS O BODE LTDA,2013-12-07 00:00:00.0,5303369,209.22


In [21]:
bode_meals_top20 = bode_meals_top20.assign(link_jarbas_consult=bode_meals_top20. \
                                         apply(func=create_jarbas_consult_document_id, axis=1))

print('Here are the links for those top 20')

for _ in range(20):
    print(bode_meals_top20.iloc[_]['link_jarbas_consult'])

Here are the links for those top 20
https://jarbas.serenatadeamor.org/#/documentId/2006450
https://jarbas.serenatadeamor.org/#/documentId/1905263
https://jarbas.serenatadeamor.org/#/documentId/1882756
https://jarbas.serenatadeamor.org/#/documentId/5476790
https://jarbas.serenatadeamor.org/#/documentId/2249894
https://jarbas.serenatadeamor.org/#/documentId/2249881
https://jarbas.serenatadeamor.org/#/documentId/5061745
https://jarbas.serenatadeamor.org/#/documentId/2249896
https://jarbas.serenatadeamor.org/#/documentId/2385459
https://jarbas.serenatadeamor.org/#/documentId/5303369
https://jarbas.serenatadeamor.org/#/documentId/5247138
https://jarbas.serenatadeamor.org/#/documentId/2415343
https://jarbas.serenatadeamor.org/#/documentId/5368496
https://jarbas.serenatadeamor.org/#/documentId/2244667
https://jarbas.serenatadeamor.org/#/documentId/2347164
https://jarbas.serenatadeamor.org/#/documentId/2415342
https://jarbas.serenatadeamor.org/#/documentId/2347168
https://jarbas.serenatadeamor

## Gas usage

In [22]:
fuel = dataset[dataset['subquota_description'] == 'Fuels and lubricants']
fuel = fuel.reset_index()
fuel.loc[:,'issue_date'] = pd.to_datetime(fuel['issue_date'], errors='coerce')

In [23]:
fuel['issue_date_day'] = fuel['issue_date'].apply(lambda date: date.day)
fuel['issue_date_month'] = fuel['issue_date'].apply(lambda date: date.month)
fuel['issue_date_year'] = fuel['issue_date'].apply(lambda date: date.year)

In [24]:
fuel.head()

,index,year,applicant_id,document_id,reimbursement_value_total,total_net_value,reimbursement_numbers,congressperson_name,congressperson_id,congressperson_document,...,net_values,month,installment,passenger,leg_of_the_trip,batch_number,reimbursement_values,issue_date_day,issue_date_month,issue_date_year
0,0,2009,1001,1564212,NaN,130.00,2888,DILCEU SPERAFICO,73768,444.0,...,130.00,4,0,NaN,NaN,388810,NaN,6.0,4.0,2009.0
1,3,2009,1001,1568056,NaN,170.00,2894,DILCEU SPERAFICO,73768,444.0,...,170.00,4,0,NaN,NaN,390158,NaN,14.0,4.0,2009.0
2,4,2009,1001,1568098,NaN,100.11,2894,DILCEU SPERAFICO,73768,444.0,...,100.11,4,0,NaN,NaN,390158,NaN,9.0,4.0,2009.0
3,7,2009,1001,1571739,NaN,93.40,2896,DILCEU SPERAFICO,73768,444.0,...,93.40,3,0,NaN,NaN,391417,NaN,6.0,3.0,2009.0
4,8,2009,1001,1571742,NaN,116.53,2896,DILCEU SPERAFICO,73768,444.0,...,116.53,3,0,NaN,NaN,391417,NaN,20.0,3.0,2009.0


In [25]:
keys = ['applicant_id', 'congressperson_id',
        'congressperson_name','issue_date_month',
        'issue_date_year']
fuel_grouped = fuel.groupby(keys)
fuel_aggregation = fuel_grouped['total_net_value'].agg({'sum': np.sum, 'expenses': len})
fuel_aggregation['expenses'] = fuel_aggregation['expenses'].astype(np.int)
fuel_aggregation = fuel_aggregation.sort_values(['sum','expenses'], ascending=[False, False])

In [26]:
fa = pd.DataFrame(fuel_aggregation.reset_index())
fa = fa.assign(link_jarbas_consult="")
fa = fa.assign(issue_date_year=fa['issue_date_year'].apply(func=np.int))
fa = fa.assign(issue_date_month=fa['issue_date_month'].apply(func=np.int))

In [27]:
fa.head(20)

,applicant_id,congressperson_id,congressperson_name,issue_date_month,issue_date_year,sum,expenses,link_jarbas_consult
0,1936,141541,SÉRGIO PETECÃO,11,2010,17801.90,4,
1,3053,178838,CÉSAR MESSIAS,10,2016,15585.41,13,
2,3041,178997,MOSES RODRIGUES,10,2016,15451.82,6,
3,3041,178997,MOSES RODRIGUES,7,2016,13798.35,6,
4,1933,141522,SABINO CASTELO BRANCO,12,2011,13500.00,3,
5,1973,4930,SILVIO COSTA,10,2010,12290.20,3,
6,1133,73781,NELSON MEURER,7,2016,11988.32,2,
7,3016,178864,ADAIL CARNEIRO,12,2016,11932.72,2,
8,2968,178853,CAETANO,10,2016,11840.00,2,
9,3053,178838,CÉSAR MESSIAS,3,2016,11677.07,8,


According to CEAP's Ato de Mesa document, it is only allowed to spend 6k of reais with fuels and lubricants:
>IX - combustíveis e lubrificantes, até o limite inacumulável de R$ 6.000,00 (seis mil reais) mensais; (Inciso com 
redação dada pelo Ato da Mesa no 49, de 3/9/2015)

In [28]:
def create_jarbas_consult_fuel_and_lubricants(row):
    jarbas_api = "https://jarbas.serenatadeamor.org/#/applicantId"
    link = [jarbas_api, row['applicant_id'],
         'month', str(int(row['issue_date_month'])),
         'subquotaId', '3',
         'year', str(int(row['issue_date_year']))]
    return '/'.join(link)

### above the 6k gas quota
Here I'll point the same 20 cases displayed above. Those are the top 20 highiest values spent on fuel

Here are the Jarbas search that lead to the top 20 above mentioned

In [29]:
fa_above6k = fa.copy()
fa_above6k = fa_above6k.assign(link_jarbas_consult=fa_above6k. \
                               apply(func=create_jarbas_consult_fuel_and_lubricants, axis=1))
for _ in range(20):
    print(fa_above6k.iloc[_]['link_jarbas_consult'])

https://jarbas.serenatadeamor.org/#/applicantId/1936/month/11/subquotaId/3/year/2010
https://jarbas.serenatadeamor.org/#/applicantId/3053/month/10/subquotaId/3/year/2016
https://jarbas.serenatadeamor.org/#/applicantId/3041/month/10/subquotaId/3/year/2016
https://jarbas.serenatadeamor.org/#/applicantId/3041/month/7/subquotaId/3/year/2016
https://jarbas.serenatadeamor.org/#/applicantId/1933/month/12/subquotaId/3/year/2011
https://jarbas.serenatadeamor.org/#/applicantId/1973/month/10/subquotaId/3/year/2010
https://jarbas.serenatadeamor.org/#/applicantId/1133/month/7/subquotaId/3/year/2016
https://jarbas.serenatadeamor.org/#/applicantId/3016/month/12/subquotaId/3/year/2016
https://jarbas.serenatadeamor.org/#/applicantId/2968/month/10/subquotaId/3/year/2016
https://jarbas.serenatadeamor.org/#/applicantId/3053/month/3/subquotaId/3/year/2016
https://jarbas.serenatadeamor.org/#/applicantId/2916/month/11/subquotaId/3/year/2016
https://jarbas.serenatadeamor.org/#/applicantId/2916/month/12/subquo

### Exactly 6k spent on fuel

Here are the top 20 most recent reimbursements that ask for exactly 6,000 reais.

In [30]:
fa_exc6k = fa.sort_values('issue_date_year', ascending=False).query('sum == 6000').head(20).reset_index()
fa_exc6k.head(20)

,index,applicant_id,congressperson_id,congressperson_name,issue_date_month,issue_date_year,sum,expenses,link_jarbas_consult
0,757,995,74537,CLAUDIO CAJADO,2,2017,6000.0,1,
1,751,958,74212,ANÍBAL GOMES,2,2017,6000.0,1,
2,688,1829,141428,FÁBIO FARIA,2,2017,6000.0,1,
3,687,1829,141428,FÁBIO FARIA,1,2017,6000.0,1,
4,677,1706,73931,ZÉ GERALDO,2,2017,6000.0,1,
5,700,1874,141483,LINDOMAR GARÇON,1,2017,6000.0,1,
6,749,958,74212,ANÍBAL GOMES,1,2017,6000.0,1,
7,743,3059,137070,VICENTINHO JÚNIOR,1,2017,6000.0,1,
8,723,2336,160576,CÉSAR HALUM,3,2017,6000.0,1,
9,722,2336,160576,CÉSAR HALUM,2,2017,6000.0,1,


Here are the links for these top 20

In [31]:
fa_exc6k = fa.copy()
fa_exc6k = fa_exc6k.assign(link_jarbas_consult=fa_exc6k. \
                           apply(func=create_jarbas_consult_fuel_and_lubricants, axis=1))
for _ in range(20):
    print(fa_exc6k.iloc[_]['link_jarbas_consult'])

https://jarbas.serenatadeamor.org/#/applicantId/1936/month/11/subquotaId/3/year/2010
https://jarbas.serenatadeamor.org/#/applicantId/3053/month/10/subquotaId/3/year/2016
https://jarbas.serenatadeamor.org/#/applicantId/3041/month/10/subquotaId/3/year/2016
https://jarbas.serenatadeamor.org/#/applicantId/3041/month/7/subquotaId/3/year/2016
https://jarbas.serenatadeamor.org/#/applicantId/1933/month/12/subquotaId/3/year/2011
https://jarbas.serenatadeamor.org/#/applicantId/1973/month/10/subquotaId/3/year/2010
https://jarbas.serenatadeamor.org/#/applicantId/1133/month/7/subquotaId/3/year/2016
https://jarbas.serenatadeamor.org/#/applicantId/3016/month/12/subquotaId/3/year/2016
https://jarbas.serenatadeamor.org/#/applicantId/2968/month/10/subquotaId/3/year/2016
https://jarbas.serenatadeamor.org/#/applicantId/3053/month/3/subquotaId/3/year/2016
https://jarbas.serenatadeamor.org/#/applicantId/2916/month/11/subquotaId/3/year/2016
https://jarbas.serenatadeamor.org/#/applicantId/2916/month/12/subquo